In [ ]:
import numpy as np
import plotly.graph_objects as go
# import soundfile as sf
import wave
import sys

show_time_plot = False

# Open the wav file
file_path = '/home/user/data/sound_test_with_chirp_20221222.wav'
                    

# Open the wav file
wav_file = wave.open(file_path, 'r')

# Extract Raw Audio from Wav File
signal = wav_file.readframes(-1)
signal = np.frombuffer(signal, dtype=np.int16)

# Get the number of channels
channels = wav_file.getnchannels()

# Split the data into channels
if channels == 2:
    signal = np.reshape(signal, (-1, 2))
    sc_output = signal[:, 0]
    sc_input = signal[:, 1]
else:
    print('The file does not have 2 channels')
    sys.exit(0)
# get the sample rate of the wav file
sample_rate = wav_file.getframerate()
 


# Create interactive plot
if show_time_plot:
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=sc_output, mode='lines', name='Sound Card Output'))
    fig.add_trace(go.Scatter(y=sc_input, mode='lines', name='Sound Card Input'))
    fig.update_layout(title='Channels Plot', xaxis_title='Sample Index', yaxis_title='Amplitude')
    fig.show()



In [ ]:
if show_time_plot:
    # cut chirp signal
    chirp = sc_output[44304:132701]

    # plot the chirp signal

    fig = go.Figure()
    fig.add_trace(go.Scatter(y=chirp, mode='lines', name='Chirp Signal'))
    fig.update_layout(title='Chirp Signal Plot', xaxis_title='Sample Index', yaxis_title='Amplitude')
    fig.show()


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

# Example data
# sc_output = np.random.randn(10000)
# sc_input = np.random.randn(10000)

# # Generate spectrogram for sc_output
# frequencies_output, times_output, Sxx_output = spectrogram(sc_output, fs=sample_rate)
# plt.figure(figsize=(10, 4))
# plt.pcolormesh(times_output, frequencies_output, 10 * np.log10(Sxx_output), shading='gouraud')
# plt.colorbar(label='Intensity [dB]')
# plt.title('Spectrogram of sc_output')
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [s]')
# plt.show()

# # Generate spectrogram for sc_input
# frequencies_input, times_input, Sxx_input = spectrogram(sc_input, fs=sample_rate)
# plt.figure(figsize=(10, 4))
# plt.pcolormesh(times_input, frequencies_input, 10 * np.log10(Sxx_input), shading='gouraud')
# plt.colorbar(label='Intensity [dB]')
# plt.title('Spectrogram of sc_input')
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [s]')
# plt.show()

# Generate spectrogram for sc_output
frequencies_output, times_output, Sxx_output = spectrogram(sc_output, fs=sample_rate)
fig_output = go.Figure(data=go.Heatmap(
    z=10 * np.log10(Sxx_output),
    x=times_output,
    y=frequencies_output,
    colorscale='Viridis'
))
fig_output.update_layout(
    title='Spectrogram of sc_output',
    xaxis_title='Time [s]',
    yaxis_title='Frequency [Hz]',
    coloraxis_colorbar=dict(title='Intensity [dB]')
)
fig_output.show()

# Generate spectrogram for sc_input
frequencies_input, times_input, Sxx_input = spectrogram(sc_input, fs=sample_rate)
fig_input = go.Figure(data=go.Heatmap(
    z=10 * np.log10(Sxx_input),
    x=times_input,
    y=frequencies_input,
    colorscale='Viridis'
))
fig_input.update_layout(
    title='Spectrogram of sc_input',
    xaxis_title='Time [s]',
    yaxis_title='Frequency [Hz]',
    coloraxis_colorbar=dict(title='Intensity [dB]')
)
fig_input.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert
import wave
import sys


# Compute the analytic signal for sc_output
analytic_signal_output = hilbert(sc_output)
instantaneous_phase_output = np.unwrap(np.angle(analytic_signal_output))
delta_phase_output = np.diff(instantaneous_phase_output)

# Compute the analytic signal for sc_input
analytic_signal_input = hilbert(sc_input)
instantaneous_phase_input = np.unwrap(np.angle(analytic_signal_input))
delta_phase_input = np.diff(instantaneous_phase_input)

# Plot the instantaneous phase (both signals in one figure)
fig1 = go.Figure()
fig1.add_trace(go.Scatter(y=instantaneous_phase_output, mode='lines', name='sc_output'))
fig1.add_trace(go.Scatter(y=instantaneous_phase_input, mode='lines', name='sc_input'))
fig1.update_layout(title='Instantaneous Phase',
                   xaxis_title='Sample',
                   yaxis_title='Phase [radians]')
fig1.show()

# Plot the change in phase angle (both signals in one figure)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(y=delta_phase_output, mode='lines', name='Change in Phase (sc_output)'))
fig2.add_trace(go.Scatter(y=delta_phase_input, mode='lines', name='Change in Phase (sc_input)'))
fig2.update_layout(title='Change in Phase Angle',
                   xaxis_title='Sample',
                   yaxis_title='Change in Phase [radians]')
fig2.show()


# Assuming delta_phase_output and delta_phase_input are numpy arrays or lists
num_samples = len(delta_phase_output)
time_axis = np.arange(num_samples) / sample_rate

# Plot the change in phase angle (both signals in one figure)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=time_axis, y=delta_phase_output, mode='lines', name='Change in Phase (sc_output)'))
fig2.add_trace(go.Scatter(x=time_axis, y=delta_phase_input, mode='lines', name='Change in Phase (sc_input)'))
fig2.update_layout(title='Change in Phase Angle',
                   xaxis_title='Time [seconds]',
                   yaxis_title='Change in Phase [radians]')
fig2.show()



In [ ]:
sc_output_start_index = 132701
sc_input_start_index = (63391-600)

# Cut the signals to the same length
num_samples = min(len(sc_output) - sc_output_start_index, len(sc_input) - sc_input_start_index)
sc_output_cut = sc_output[sc_output_start_index:sc_output_start_index + num_samples]
sc_input_cut = sc_input[sc_input_start_index:sc_input_start_index + num_samples]

# Compute the analytic signal for sc_output_cut
analytic_signal_output_cut = hilbert(sc_output_cut)
instantaneous_phase_output_cut = np.unwrap(np.angle(analytic_signal_output_cut))
delta_phase_output_cut = np.diff(instantaneous_phase_output_cut)

# Compute the analytic signal for sc_input_cut
analytic_signal_input_cut = hilbert(sc_input_cut)
instantaneous_phase_input_cut = np.unwrap(np.angle(analytic_signal_input_cut))
delta_phase_input_cut = np.diff(instantaneous_phase_input_cut)

# Plot the change in phase angle (both signals in one figure)
fig3 = go.Figure()
fig3.add_trace(go.Scatter(y=delta_phase_output_cut, mode='lines', name='Change in Phase (sc_output)'))
fig3.add_trace(go.Scatter(y=delta_phase_input_cut, mode='lines', name='Change in Phase (sc_input)'))
fig3.update_layout(title='Change in Phase Angle (Cut Signals)',
                   xaxis_title='Sample',
                   yaxis_title='Change in Phase [radians]')

In [ ]:
run_correlation =False
if run_correlation:
    # print delay in samples and seconds
    [delay_samples, delay_seconds]=find_delay(sc_output, sc_input, sample_rate)
    print(f"Estimated delay: {delay_samples} samples, which is {delay_seconds:.5f} seconds.")

    # Find the delay using correlation
    correlation = np.correlate(sc_output, sc_input, mode='full')
    delay = np.argmax(correlation) - (len(sc_output) - 1)
    print(f'Delay between channels: {delay} samples')

    # plot the correlation
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=correlation, mode='lines', name='Correlation'))
    fig.update_layout(title='Correlation Plot', xaxis_title='Sample Index', yaxis_title='Correlation')
    fig.show()



    # Create interactive plot
    fig = go.Figure()
    if delay < 0:
        fig.add_trace(go.Scatter(y=sc_output, mode='lines', name='Sound Card Output'))
        fig.add_trace(go.Scatter(y=sc_input[(-delay):], mode='lines', name='Sound Card Input'))
    else:
        fig.add_trace(go.Scatter(y=sc_output[delay:], mode='lines', name='Sound Card Output'))
        fig.add_trace(go.Scatter(y=sc_input, mode='lines', name='Sound Card Input'))
    fig.update_layout(title='Aligned Channels Plot', xaxis_title='Sample Index', yaxis_title='Amplitude')
    fig.show()
